In [15]:
from esco import LocalDB
db = LocalDB()
skills_labels = [
        "collaborate with engineers",
        "deploy cloud resource",
        "design cloud architecture",
        "design cloud networks",
        "plan migration to cloud",
        "automate cloud tasks",
        "coordinate engineering teams",
        "design database in the cloud",
        "design for organisational complexity",
        "develop with cloud services",
        "do cloud refactoring",
    ]
skills = db.skills[db.skills.label.str.lower().isin(skills_labels)]
labels  = [l for labels in skills.allLabel for l in labels]

In [70]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_trf")
# nlp.add_pipe("merge_noun_chunks")
def find_root(doc):
    for prefix in ("", "to "):
        doc = nlp(prefix + doc.text)
        for token in doc:
            if token.dep_ == "ROOT" and token.pos_ == "VERB":
                return token
    return None


def find_obj(token):
    for child in token.children:
        if child.dep_ in ("prep"):
            return find_obj(child)

        if child.dep_ in ("dobj", "pobj", "nsubj"):
            return child
    return None

def find_compound(token):
    if token:
        yield token.lemma_
        if token.children:
            for child in token.children:
                if child.dep_ == "compound":
                    yield from find_compound(child)

In [71]:
# Print the couples (verb, dobj or pobj) for each doc in docs.
docs = nlp.pipe(labels)
for doc in docs:
    verb = None
    dobj = None
    pobj = None
    for token in doc:
       verb = find_root(doc)
    if not verb:
        print("****missing verb", doc.text)
        continue
    displacy.render(verb.doc, style="dep", options={"compact": True})
    obj = find_obj(verb)
    print(verb.lemma_, list(find_compound(obj)), f"-{doc.text}-")

plan
['refactoring']
-plan refactoring-

plan
['migration']
-plan migration to cloud-

****missing verb cloud migration planning

design
['architecture', 'cloud']
-design multi-tier cloud architecture-

create
['architecture', 'cloud']
-create cloud architecture-

...

design
['database']
-design database in the cloud-

automate
['task', 'cloud']
-automate cloud tasks-

****missing verb automation of cloud tasks

design
['organisation']
-design cloud environment for complex organisations-

design
['complexity']
-design for organisational complexity-

In [44]:
t="design multi-tier cloud architecture"
doc=nlp(t)

In [45]:
r = find_root(doc)

In [49]:
a, b =list(r.children)

In [53]:
[print(token.text, token.dep_) for token in b.children]

multi amod

- amod

tier amod

cloud compound

[None, None, None, None]